In [31]:
import numpy as np
import time
from matplotlib import pyplot as plt
import NEON_AOP_waveform_tools # written by Tristan, lots of functions
import pandas as pd

In [2]:
# Define the PLS file of the pulsewaves data
PlsFile = r"2015032618_Q780_150326_194954_1_originalpoints.pls"
# there are 2 million pulse points
# You have to have the .wvs available as well.

In [3]:
# Figure out how many pulses there are
fin_pls_file = open(PlsFile,"rb")
fin_pls_file.seek(184,0)
number_of_pulses = np.fromfile(fin_pls_file,np.int64,1)
number_of_pulses

array([24063095])

In [6]:
# Define the pulses we want 
pulses = np.arange(0,10)

In [29]:
for pulse in pulses:
    pulse=[pulse]
    # Read in pulse number info
    # 1) coordinate info
    # 2) received wave
    # 3) outgoing wave
    coord_info, wave_in, wave_out = NEON_AOP_waveform_tools.waveform_read(pulse,PlsFile)
    # print coordinates
    #print(coord_info) # first number is GPS time, next 3 describe the direction of the (3d coordinates), 
    # last three number are the actual coordinates the start of the pulse (x,y,z)
    
    initial_params_out = NEON_AOP_waveform_tools.get_params(wave_out[0])
    initial_params_in = NEON_AOP_waveform_tools.get_params(wave_in[0])
    
    num_peaks_in = np.array(initial_params_in[0,-1]) # number of peaks is saved in this location

    WFtime = np.arange(1,244) # set up time (ns bins)
    
    fit_wave_final = NEON_AOP_waveform_tools.get_fit_wave(initial_params_in, 244, num_peaks_in)
    
    FWHM = (2.355*(initial_params_in[0,2:int(num_peaks_in*3):3]/2**0.5))/2 # full width half max. Grab 3rd, 6th, 9th parameters
    
    trig_loc = initial_params_in[0,1:int(num_peaks_in*3):3] - FWHM
    trig_loc  = np.reshape(trig_loc,(int(num_peaks_in),1))
    
    # normalize trigger locations by the location of the outgoing pulse
    trig_loc = initial_params_in[0,1:int(num_peaks_in*3):3]-FWHM-initial_params_out[0,1]
    trig_loc = np.reshape(trig_loc,(int(num_peaks_in),1))
    
    pulse_coords = (np.matlib.repmat(trig_loc[:,0],3,1)).transpose()*coord_info[:,1:4]+coord_info[:,4:7]
    print(pulse_coords)
    

read 0 pulses in file in 7.80000000020209e-05 seconds, we are [0.]% done
[[5.31524626e+05 4.31784554e+06 2.26042556e+03]
 [5.31524213e+05 4.31784563e+06 2.25898804e+03]]
[[5.31525002e+05 4.31784554e+06 2.26030634e+03]
 [5.31524589e+05 4.31784563e+06 2.25886865e+03]]
[[5.31525982e+05 4.31784506e+06 2.26204872e+03]
 [5.31525447e+05 4.31784518e+06 2.26017949e+03]
 [5.31525035e+05 4.31784527e+06 2.25874162e+03]]
[[5.31524272e+05 4.31784455e+06 2.26020398e+03]
 [5.31523859e+05 4.31784464e+06 2.25876656e+03]
 [5.31523528e+05 4.31784472e+06 2.25761663e+03]]
[[5.31524689e+05 4.31784454e+06 2.26022167e+03]
 [5.31524235e+05 4.31784464e+06 2.25864031e+03]
 [5.31523946e+05 4.31784471e+06 2.25763399e+03]]
[[5.31525213e+05 4.31784461e+06 2.26023246e+03]
 [5.31524760e+05 4.31784471e+06 2.25865089e+03]
 [5.31524389e+05 4.31784479e+06 2.25735688e+03]]
[[5.31525566e+05 4.31784457e+06 2.26011263e+03]
 [5.31525113e+05 4.31784467e+06 2.25853087e+03]
 [5.31524743e+05 4.31784475e+06 2.25723671e+03]]
[[5.3152